# Apache Kafkaのセットアップ

Apache Kafka の broker をセットアップします。

# 事前チェック

Apache Kafkaの構築を行うノードは「00-001-ノードの起動.ipynb」の Notebook で起動したVCノードである、または「00-002-既存ノードの登録」で準備したノードであることを想定しています。

それ以外のノードに対して環境構築を行う場合は、以下の条件を満たすように準備を行ってください。

1. 対象となるノードを Ansible で操作できる
1. Docker のセットアップが済んでいる
1. 構築に利用するツールがインストールされていること
  - rsync
  - docker-compose
1. ホスト名が名前解決できること

1., 2., 3. については、以下でチェックします。

## Ansible

In [ ]:
target = 'broker'

Ansibleで操作できることを確認します。

In [ ]:
!ansible {target} -m ping

In [ ]:
!ansible {target} -b -a 'whoami'

## Docker

Dockerが利用できることを確認します。

In [ ]:
!ansible {target} -a 'docker info'

## ツールなど

必要なコマンドがインストールされていることを確認します。

In [ ]:
!ansible {target} -a 'which rsync'

In [ ]:
!ansible {target} -a 'docker-compose version'

# パラメータの指定

kafka broker のホスト名のリストを次のセルで設定してください。

In [ ]:
hostnames = [
    'broker-0',
    'broker-1',
    'broker-2',
]

kafka broker をAWS VPCの外側のネットワークからアクセスする場合は、次のセルで外部に公開するホスト名を次のセルで設定してください。

In [ ]:
# ext_hostnames = [
#    'broker-0.example.org',
#    'broker-1.example.org',
#    'broker-2.example.org',
#]

kafka broker のポート番号のリストを次のセルで設定してください。

In [ ]:
kafka_port = [9092 for x in range(len(hostnames))]

kafka broker をAWS VPCの外側のネットワークからアクセスする場合は、次のセルで外部に公開するポート番号を次のセルで設定してください。

> 外部からKafka broker にアクセスできるようにするためにはAWSのセキュリティグループ設定やNATインスタンスの設定を変更する必要があります。

In [ ]:
# ext_kafka_port = [
#    19092,
#    19093,
#    19094,
#]

# ZooKeeperの実行

ZooKeeper をコンテナで実行します。

各VCノードで ZooKeeperのコンテナを実行するための `docker-compose.yml` を作成します。

* コンテナイメージ
  - [zookeeper](https://hub.docker.com/_/zookeeper)
* ZooKeeper バージョン
  - 3.4
* ネットワーク
  - ホストネットワーク

In [ ]:
from tempfile import TemporaryDirectory
from pathlib import Path
import json

with TemporaryDirectory() as work_dir:
    docker_compose_yml = Path(work_dir) / 'docker-compose.yml'
    with docker_compose_yml.open(mode='w') as f:
        f.write('''version: '2.4'
services:
  zoo{{my_id}}:
    image: zookeeper:3.4
    environment:
      ZOO_MY_ID: {{my_id}}
      ZOO_SERVERS:{% for sv in hostnames %} server.{{loop.index}}={{sv}}:2888:3888{% endfor %}

    network_mode: 'host'
    init: true
    restart: always
    volumes:
      - "./data:/data"
      - "./datalog:/datalog"
''')
    !ansible {target} -m file -a 'path=zk state=directory'
    for x in range(len(hostnames)):
        params = {
            'my_id': x + 1,
            'hostnames': hostnames,
        }
        params_json = Path(work_dir) / 'params.json'
        with params_json.open(mode='w') as f:
            json.dump(params, f)
        !ansible {hostnames[x]} -m template -e @{params_json} \
            -a 'src={docker_compose_yml} dest=zk/'

コンテナを実行します。

In [ ]:
!ansible {target} -a 'chdir=zk docker-compose up -d'

コンテナの状態を確認します。

In [ ]:
!ansible {target} -a 'chdir=zk docker-compose ps'

# Apache Kafka の実行

Apache Kafka をコンテナで実行します。

各VCノードで Apache Kafka のコンテナを実行するための `docker-compose.yml` を作成します。

* コンテナイメージ
  - [wurstmeister/kafka](https://hub.docker.com/r/wurstmeister/kafka)
* Apache Kafka バージョン
  - 2.2.1
  - Scala: 2.12
* ネットワーク
  - ホストネットワーク
  
* brokerリスナ
  - 外部用(NATインスタンス経由)

In [ ]:
with TemporaryDirectory() as work_dir:
    docker_compose_yml = Path(work_dir) / 'docker-compose.yml'
    with docker_compose_yml.open(mode='w') as f:
        f.write('''version: '2.4'
services:
  broker{{broker_id}}:
    image: wurstmeister/kafka:2.12-2.2.1
    environment:
      KAFKA_BROKER_ID: "{{broker_id}}"
      KAFKA_ZOOKEEPER_CONNECT: {% for sv in hostnames %}{%if not loop.first %},{% endif %}{{sv}}:2181{% endfor %}

{% if ext_hostnames is defined %}
      KAFKA_ADVERTISED_LISTENERS: INSIDE://{{hostnames[broker_id]}}:{{kafka_port[broker_id]}},OUTSIDE://{{ext_hostnames[broker_id]}}:{{ext_kafka_port[broker_id]}}
      KAFKA_LISTENERS: INSIDE://:{{kafka_port[broker_id]}},OUTSIDE://:{{ext_kafka_port[broker_id]}}
      KAFKA_LISTENER_SECURITY_PROTOCOL_MAP: INSIDE:PLAINTEXT,OUTSIDE:PLAINTEXT
      KAFKA_INTER_BROKER_LISTENER_NAME: INSIDE
{% else %}
      KAFKA_ADVERTISED_LISTENERS: INSIDE://{{hostnames[broker_id]}}:{{kafka_port[broker_id]}}
      KAFKA_LISTENERS: INSIDE://:{{kafka_port[broker_id]}}
      KAFKA_LISTENER_SECURITY_PROTOCOL_MAP: INSIDE:PLAINTEXT
      KAFKA_INTER_BROKER_LISTENER_NAME: INSIDE
{% endif %}
    network_mode: 'host'
    init: true
    restart: always
    volumes:
      - "./kafka:/kafka"
''')
    !ansible {target} -m file -a 'path=kafka state=directory'
    for x in range(len(hostnames)):
        params = {
            'broker_id': x,
            'hostnames': hostnames,
            'kafka_port': kafka_port,
        }
        if 'ext_hostnames' in globals():
            params['ext_hostnames'] = ext_hostnames
        if 'ext_kafka_port' in globals():
            params['ext_kafka_port'] = ext_kafka_port
        params_json = Path(work_dir) / 'params.json'
        with params_json.open(mode='w') as f:
            json.dump(params, f)
        !ansible {hostnames[x]} -m template -e @{params_json} \
            -a 'src={docker_compose_yml} dest=kafka/'

コンテナを実行します。

In [ ]:
!ansible {target} -a 'chdir=kafka docker-compose up -d'

コンテナの状態を確認します。

In [ ]:
!ansible {target} -a 'chdir=kafka docker-compose ps'